In [21]:
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

In [13]:
mask = load_model("model_aug.h5")
gender = load_model("man_woman.h5")
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Ayol yoki erkakning maskasi bor, yo'qligini topuvchi dastur

In [22]:
cap = cv2.VideoCapture(0)

while True:  
    ret, img = cap.read()  
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray_img, 1.25, 4, minSize=(10, 10), flags=cv2.CASCADE_SCALE_IMAGE) 
    
    if len(faces) != 0:
        for (x,y,w,h) in faces: 
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)  
            rec_gray = gray_img[y:y+h, x:x+w] 
            rec_color = img[y:y+h, x:x+w, :]
            
            rec_color = cv2.resize(rec_color,(224, 224), interpolation=cv2.INTER_CUBIC)
            rec_color = preprocess_input(rec_color)
            gen_res = gender.predict(np.expand_dims(rec_color, 0), verbose=0)
            
            if np.round(gen_res[0][0]) == 1:
                mask_res = mask.predict(np.expand_dims(rec_color, 0), verbose=0)
                if np.round(mask_res[0][0]) == 1:
                    status = "Maskali Erkak"
                else:
                    status = "Maskasiz Erkak"                
            else:
                mask_res = mask.predict(np.expand_dims(rec_color, 0), verbose=0)
                if np.round(mask_res[0][0]) == 1:
                    status = "Maskali Ayol"
                else:
                    status = "Maskasiz Ayol" 
            img = cv2.putText(img, status, (x,y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
            
    else:
        maska_statusi = "Yuz topilmadi!"
        img = cv2.putText(img, maska_statusi, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
    cv2.imshow("Mask detection",img) 
  
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release() 
cv2.destroyAllWindows()

# Cascade faylisiz yuzni topuvchi dastur

In [28]:
import cv2
detector = cv2.HOGDescriptor()
detector.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (face, _) = detector.detectMultiScale(gray, 1.1, 2, minSize=(5, 5), flags = cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in face:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.imshow("Yuz", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == "q":
            break
        
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'detectMultiScale'
> Overload resolution failed:
>  - 'minSize' is an invalid keyword argument for HOGDescriptor.detectMultiScale()
>  - 'minSize' is an invalid keyword argument for HOGDescriptor.detectMultiScale()
